In [1]:
import pennylane as qml
import numpy as np
import tensorflow as tf

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
dev = qml.device('cirq.simulator', wires=3)

In [4]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

In [64]:
def generator(w, **kwargs): # what does kwargs do
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [65]:
@qml.qnode(dev, interface='tf')
def real_disc_circuit(phi, theta, omega, disc_weights):
    # what are these varaibles?
    # phi, theta, and omega are the real rotational values
    # disc_weights are discriminator weights
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2)) # measure the expval of wire 2 in the z basis, the discriminator guess of wehther it's real or not

@qml.qnode(dev, interface='tf')
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2)) #same: return discriminator guess

In [87]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)

    # why does this convert into probability?
    prob_real_true = (true_disc_output + 1) / 2
    print('Circuit output expval: {} Probability that the generator classifies it as real {}'.format(true_disc_output, prob_real_true))
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    prob_fake_true = (fake_disc_output + 1 ) / 2
    return prob_fake_true

def gen_cost(gen_weights):
    cost = -prob_fake_true(gen_weights, disc_weights)
    return cost

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost 

In [105]:
phi = np.pi/6
theta = np.pi/2
omega = np.pi/7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)
# gen_weights = init_gen_weights
# disc_weights = init_disc_weights
print(init_gen_weights, init_disc_weights, gen_weights)
print(type(init_gen_weights), type(gen_weights))
print(gen_weights[0].numpy())
print(gen_weights[0])

[ 3.15923318e+00  4.00157208e-03  9.78737984e-03  2.24089320e-02
  1.86755799e-02 -9.77277880e-03  9.50088418e-03 -1.51357208e-03
 -1.03218852e-03] [ 0.4105985   0.14404357  1.45427351  0.76103773  0.12167502  0.44386323
  0.33367433  1.49407907 -0.20515826] <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 3.15923318e+00,  4.00157208e-03,  9.78737984e-03,  2.24089320e-02,
        1.86755799e-02, -9.77277880e-03,  9.50088418e-03, -1.51357208e-03,
       -1.03218852e-03])>
<class 'numpy.ndarray'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
3.1592331770494697
tf.Tensor(3.1592331770494697, shape=(), dtype=float64)


In [102]:
opt = tf.keras.optimizers.SGD(0.4)

In [90]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))
        print("Prob(real classified as real):", prob_real_true(disc_weights).numpy())
        print("Prob(fake classified as real):", prob_fake_true(gen_weights, disc_weights).numpy())

Circuit output expval: -0.6303413510322571 Probability that the generator classifies it as real 0.18482932448387146
Circuit output expval: -0.542709693312645 Probability that the generator classifies it as real 0.22864515334367752
Step 0: cost = -0.05727687478065491
Circuit output expval: -0.542709693312645 Probability that the generator classifies it as real 0.22864515334367752
Prob(real classified as real): 0.22864515334367752
Prob(fake classified as real): 0.1713682785630226
Circuit output expval: -0.542709693312645 Probability that the generator classifies it as real 0.22864515334367752
Circuit output expval: -0.4338969588279724 Probability that the generator classifies it as real 0.2830515205860138
Circuit output expval: -0.30347228050231934 Probability that the generator classifies it as real 0.34826385974884033
Circuit output expval: -0.15359419584274292 Probability that the generator classifies it as real 0.42320290207862854
Circuit output expval: 0.010194838047027588 Probabili

In [103]:
cost = disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost()
        print("Step {}: cost = {}".format(step, cost_val))
        print("Prob(real classified as real):", prob_real_true(disc_weights))
        print("Prob(fake classified as real):", prob_fake_true(gen_weights, disc_weights))

Circuit output expval: -0.6303413510322571 Probability that the generator classifies it as real 0.18482932448387146


ValueError: `tape` is required when a `Tensor` loss is passed.

In [83]:
print("Prob(real classified as real):", prob_real_true(disc_weights).numpy())
print("Prob(fake classified as real):", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(real classified as real): 0.9985871425596997
Prob(fake classified as real): 0.5011128038167953


In [84]:
cost = lambda: gen_cost(gen_weights)

for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.5833386033773422
Step 5: cost = -0.8915732949972153
Step 10: cost = -0.9784244522452354
Step 15: cost = -0.9946483590174466
Step 20: cost = -0.9984995491686277
Step 25: cost = -0.9995636216044659
Step 30: cost = -0.9998718172573717
Step 35: cost = -0.9999619696027366
Step 40: cost = -0.9999888275397097
Step 45: cost = -0.999996672290763


In [85]:
print(prob_fake_true(gen_weights, disc_weights).numpy())

0.99999862746688


In [108]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.2169418   0.45048445 -0.86602525]
Generator Bloch vector: [ 0.02237707  0.00048757 -0.00974897]
Generator weights, the imput to ^ [ 3.15923318e+00  4.00157208e-03  9.78737984e-03  2.24089320e-02
  1.86755799e-02 -9.77277880e-03  9.50088418e-03 -1.51357208e-03
 -1.03218852e-03]


AttributeError: 'function' object has no attribute 'numpy'